In [1]:
import spacy
import pandas as pd

In [8]:
#%%timeit -n 1 -r 1
nlp = spacy.load("en_core_web_trf")

In [9]:
nlp

In [10]:
def read_page(secure_path,f,page):
    page_format = "{:08d}".format(page)
    sfile = f"{secure_path}/{f}/{page_format}.txt"
    sentence_ner = []
    sentence_id = 0
    with open(sfile,"r") as file:
        temp_page = []
        for l in file:
            temp_page.append(l.strip())
        temp_page_str = " ".join(temp_page).replace("- ","")
        doc = nlp(temp_page_str)
        temp_sents = []
        for s in doc.sents:
            temp_sents.append((f,page,sentence_id,str(s)))
            #temp_sents.append(s)
            #[(ts,ts.label_,ts.start_char,ts.end_char,tt.text[ts.start_char-tt.start_char:ts.end_char-tt.start_char]) for ts in tt.ents]
            for eid,ts in enumerate(s.ents):
                sentence_ner.append((f,page,sentence_id,eid,str(ts),ts.label_,ts.start_char-s.start_char,ts.end_char-s.start_char))
            sentence_id+=1
    return temp_sents,sentence_ner

In [11]:
secure_path = "/media/secure_volume/ner/volumes"

In [12]:
test = read_page(secure_path,"mdp.39015041788319",20)

/home/dcuser/anaconda3/envs/ner/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [13]:
pd.DataFrame(test[1]).head(20)

,0,1,2,3,4,5,6,7
0,mdp.39015041788319,20,0,0,Colony,ORG,34,40
1,mdp.39015041788319,20,2,0,Mohegan,NORP,25,32
2,mdp.39015041788319,20,2,1,two,CARDINAL,50,53
3,mdp.39015041788319,20,2,2,John's Town,FAC,76,87
4,mdp.39015041788319,20,2,3,Ben's Town.24,FAC,111,124
5,mdp.39015041788319,20,3,0,Emma Baker,PERSON,15,25
6,mdp.39015041788319,20,3,1,Tribe,ORG,39,44
7,mdp.39015041788319,20,3,2,John Uncas,PERSON,58,68
8,mdp.39015041788319,20,3,3,Ben Uncas,PERSON,74,83
9,mdp.39015041788319,20,3,4,Mamohet,PERSON,96,103


In [14]:
%%timeit -n 1 -r 1

import os
import tqdm

for x in os.listdir(secure_path):
    result = []
    try:
        for y in tqdm.tqdm(os.listdir(f"{secure_path}/{x}")):
            if y.endswith(".txt"):
                page = int(y.split(".")[0])
                test = read_page(secure_path,x,page)
                result.extend(test[1])
            #print(y)
        pd.DataFrame(result,columns=["vol_id","page", \
                                     "sentence_id","ent_id","entity", \
                                     "entity_type","start_char","end_char"]). \
                    sort_values(["page","sentence_id","ent_id"]). \
                    to_csv(f"ner-lg-output-{x}.csv",index=False)
    except:
        continue

  0%|                                                   | 0/348 [00:00<?, ?it/s]/home/dcuser/anaconda3/envs/ner/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
  0%|                                                    | 0/68 [00:00<?, ?it/s]/home/dcuser/anaconda3/envs/ner/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
  0%|                                                   | 0/266 [00:00<?, ?it/s]/home/dcuser/anaconda3/envs/ner/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided de

1h 1min 2s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
